<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Ensemble_with_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ensemble


In [1]:
'''imports '''
from google.colab import files
uploaded = files.upload()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers


from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%pip install fasttext


# The complete datasets needed
train_df = pd.read_json('train_df_complete.json', orient="records", lines=True)
dev_df = pd.read_json('dev_df_complete.json', orient="records", lines=True)

Saving dev_df_complete.json to dev_df_complete.json
Saving train_df_complete.json to train_df_complete.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313485 sha256=ed2403c25c38b7519b55f4e0d4253a4a19c1a680ea75f3926559f57b6207b66b
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


## Import the 3 models

In [2]:
!pip install huggingface_hub tensorflow --quiet

from huggingface_hub import login, create_repo, upload_folder
import os
import tensorflow as tf
from huggingface_hub import hf_hub_download

# Repo name and model file name
repo_id = "tgarnier067/MNLP-HW1"
model_name_1 = "model_logits_features.keras"
model_name_2 = "model_logits_description.keras"
model_name_3 = "model_logits_photos.keras"

# Download the model from Hugging Face
model_path_1 = hf_hub_download(repo_id=repo_id, filename=model_name_1)
model_path_2 = hf_hub_download(repo_id=repo_id, filename=model_name_2)
model_path_3 = hf_hub_download(repo_id=repo_id, filename=model_name_3)


# Load the .keras model with TensorFlow
model_logits_features = tf.keras.models.load_model(model_path_1)
model_logits_description = tf.keras.models.load_model(model_path_2)
model_logits_photos = tf.keras.models.load_model(model_path_3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_logits_features.keras:   0%|          | 0.00/8.74M [00:00<?, ?B/s]

model_logits_description.keras:   0%|          | 0.00/9.86M [00:00<?, ?B/s]

model_logits_photos.keras:   0%|          | 0.00/517k [00:00<?, ?B/s]

## model_logits_description

### Prepare the data

In [3]:
 import fasttext.util
 fasttext.util.download_model('en', if_exists='ignore')  # English
 ft = fasttext.load_model('cc.en.300.bin')

In [4]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
from sklearn.preprocessing import normalize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
'''transforming the descriptions to the embeddings space , adding the oov information'''
def transform_description(description):
  oov_words = 0
  description = nltk.word_tokenize(description)
  description = [word for word in description if word not in stop_words]
  description = [word.lower() for word in description]
  for word in description:
    if word not in ft.words:
      oov_words += 1
  description = " ".join(description)
  description_vector = ft.get_sentence_vector(description)
  description_vector = normalize(description_vector.reshape(1, -1), norm='l2')[0]
  description_vector = np.append(description_vector, oov_words)
  return description_vector

In [6]:
# Extract text and label in train set
train_texts = train_df['description'].to_list()
train_labels = train_df['label_int'].to_list()

# Extract text and label in dev set
dev_texts = dev_df['description'].to_list()
dev_labels = dev_df['label_int'].to_list()

# Vectorize the texts with fasttext
train_vectors = np.array([transform_description(text) for text in train_texts])
dev_vectors = np.array([transform_description(text) for text in dev_texts])

### Apply the model to the data

In [7]:
logits_dev_model_descriptions = model_logits_description.predict(dev_vectors)
logits_train_model_description = model_logits_description.predict(train_vectors)

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 191ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [8]:
df_train_description = pd.DataFrame(logits_train_model_description, columns=['logit_0', 'logit_1', 'logit_2'])
df_train_description['label'] = train_df['label_int']
dev_df_description = pd.DataFrame(logits_dev_model_descriptions, columns=['logit_0', 'logit_1', 'logit_2'])
dev_df_description['label'] = dev_df['label_int']


In [9]:
'''sanity check on the dataframe'''
df_train_description.head(10)

,logit_0,logit_1,logit_2,label
0,1.905099,1.534088,-3.094239,0
1,0.760209,2.901672,-5.288945,1
2,-0.636569,2.368679,-4.468927,1
3,1.309630,1.982279,-3.745397,1
4,-1.129772,2.629756,-4.753024,1
5,1.146501,1.890622,-3.682914,1
6,0.381403,1.764994,-3.796714,0
7,1.686283,2.372820,-4.080890,1
8,-1.380255,2.684189,-4.805522,1
9,1.886965,1.921860,-3.410134,0


## model_logits_features

### Prepare the data

In [10]:
# Get the list of explicatives variables
list_var = ['subclass_depth', 'subclasses', 'instances_of', 'instances_of_up', 'date', 'nb_lang',
       'descr_num_nouns', 'descr_num_verbs',
       'descr_num_adjectives', 'descr_has_location', 'descr_has_ethnic_group',
       'descr_has_event', 'type_cat', 'category_architecture',
       'category_biology', 'category_comics and anime', 'category_fashion',
       'category_films', 'category_food', 'category_geography',
       'category_literature', 'category_media', 'category_music',
       'category_politics', 'category_sports', 'category_transportation',
       'category_visual arts', 'category_combined',
       'main_country_cat'
]

# Create the X and y, train and dev
X_train = train_df[list_var]
y_train = train_df['label_int'].values

X_dev = dev_df[list_var]
y_dev = dev_df['label_int'].values

# Selecting numerical columns from X_train (all of them are int64 or float64)
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Defining pipeline: impute Nan, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Appling pipeline to numeric columns
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols)
])


#adjustin the data to proper format
X_train_processed = preprocessor.fit_transform(X_train)
X_dev_processed = preprocessor.transform(X_dev)

# float cast
X_train_processed = X_train_processed.astype('float32')
X_dev_processed = X_dev_processed.astype('float32')

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=3)
y_dev_cat = to_categorical(y_dev, num_classes=3)

### Apply the model on the data

In [11]:
logits_dev_model_features = model_logits_features.predict(X_dev_processed)
logits_train_model_features = model_logits_features.predict(X_train_processed)

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


## model_logits_photos

In [12]:
# transform the data into arrays
train_df['image'] = train_df['image'].apply(lambda x: np.array(x))
dev_df['image'] = dev_df['image'].apply(lambda x: np.array(x))

In [13]:
# Images normalization
X_train_fill = np.stack(train_df['image'].to_numpy()).astype('float32') / 255.0
y_train_fill = train_df['label']

X_dev_fill = np.stack(dev_df['image'].to_numpy()).astype('float32') / 255.0
y_dev_fill = dev_df['label']

In [14]:
# Apply the model
logits_dev_model_photos = model_logits_photos.predict(X_dev_fill)
logits_train_model_photos = model_logits_photos.predict(X_train_fill)

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


## Ensemble model


here we try to combine two classifiers, one basing on the neural network working on features extracted frrom the wikidata/wikipedia, and the second one basing on the neural network working on the embedding representation of the description of the wikipedia/wikidata entities. We combine them with a meta lerner that as training set  takes the logits for each label from both of the trained base neural networks, achieved on the train set. And the validation are the logits of these two networks acquired by predcition on the validation set.

### Build and fit the ensemble model

In [20]:
# Prepare the data
train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = np.concatenate((logits_train_model_description, logits_train_model_features), axis=1)
train_vectors = np.concatenate((train_vectors, logits_train_model_photos), axis=1)
dev_vectors = np.concatenate((logits_dev_model_descriptions, logits_dev_model_features), axis=1)
dev_vectors = np.concatenate((dev_vectors, logits_dev_model_photos), axis=1)

# Build early stop and reduce lr condition
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)


# Define the model
model_Ensemble = models.Sequential([
    layers.Input(shape=(9,)),
    layers.Dense(16, kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(8, kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(3, activation='softmax')
])

# Compile and train
model_Ensemble.compile(optimizer=Adam(learning_rate=3e-4),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

history = model_Ensemble.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.5836 - loss: 1.0049 - val_accuracy: 0.6600 - val_loss: 0.7805 - learning_rate: 3.0000e-04
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7015 - loss: 0.7468 - val_accuracy: 0.6733 - val_loss: 0.7481 - learning_rate: 3.0000e-04
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7697 - loss: 0.6009 - val_accuracy: 0.6767 - val_loss: 0.7436 - learning_rate: 3.0000e-04
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8127 - loss: 0.5178 - val_accuracy: 0.6733 - val_loss: 0.7454 - learning_rate: 3.0000e-04
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8366 - loss: 0.4571 - val_accuracy: 0.6800 - val_loss: 0.7534 - learning_rate: 3.0000e-04
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8585 - loss: 0.4109 - val_accuracy: 0.6900 - val_loss: 0.7590 - learning_rate: 3.0000e-04
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8694 - loss: 0.3

### Evaluation

Seems overfitted to the training set, with discreancy of the accuracies between the training and the validation set.

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Prepare the data for evaluation
y_train_pred = model_Ensemble.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)

y_dev_pred = model_Ensemble.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# Compute evaluations
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)
train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(y_train_true_classes, y_train_pred_classes, average='weighted')
dev_precision, dev_recall, dev_f1, _ = precision_recall_fscore_support(y_dev_true_classes, y_dev_pred_classes, average='weighted')

# Print
print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Train precision: {train_precision * 100:.2f}%")
print(f"Train recall   : {train_recall * 100:.2f}%")
print(f"Train F1-score : {train_f1 * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")
print(f"Dev precision  : {dev_precision * 100:.2f}%")
print(f"Dev recall     : {dev_recall * 100:.2f}%")
print(f"Dev F1-score   : {dev_f1 * 100:.2f}%")

196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Train accuracy : 93.70%
Train precision: 93.68%
Train recall   : 93.70%
Train F1-score : 93.67%
Dev accuracy   : 70.67%
Dev precision  : 72.21%
Dev recall     : 70.67%
Dev F1-score   : 70.39%


### Try XGBoost, Logistic Regression and SVM

In [17]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05)

# Training
model_xgb.fit(train_vectors, train_labels)

# Prediction
y_pred = model_xgb.predict(dev_vectors)

# Evaluation
acc = accuracy_score(dev_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(dev_labels, y_pred, average='weighted')

# Print
print(f"Accuracy XGBoost : {acc:.4f}")
print(f"Precision        : {precision:.4f}")
print(f"Recall           : {recall:.4f}")
print(f"F1-score         : {f1:.4f}")


Accuracy XGBoost : 0.6567
Precision        : 0.6701
Recall           : 0.6567
F1-score         : 0.6549


In [18]:
from sklearn.linear_model import LogisticRegression

# Training
model_lr = LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs')
model_lr.fit(train_vectors, train_labels)

# Prediction
y_pred = model_lr.predict(dev_vectors)

# Evaluation
acc = accuracy_score(dev_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(dev_labels, y_pred, average='weighted')

# Print
print(f"Accuracy Logistic Regression : {acc:.4f}")
print(f"Precision                   : {precision:.4f}")
print(f"Recall                      : {recall:.4f}")
print(f"F1-score                    : {f1:.4f}")


Accuracy Logistic Regression : 0.6667
Precision                   : 0.6870
Recall                      : 0.6667
F1-score                    : 0.6661


In [19]:
from sklearn.svm import SVC

# Training
model_svm = SVC(C=1, kernel='rbf', probability=True)
model_svm.fit(train_vectors, train_labels)

# Prediction
y_pred = model_svm.predict(dev_vectors)

# Evaluation
acc = accuracy_score(dev_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(dev_labels, y_pred, average='weighted')

# Print
print(f"Accuracy SVM  : {acc:.4f}")
print(f"Precision     : {precision:.4f}")
print(f"Recall        : {recall:.4f}")
print(f"F1-score      : {f1:.4f}")

Accuracy SVM  : 0.6633
Precision     : 0.6835
Recall        : 0.6633
F1-score      : 0.6642


### Exportation of the ensemble model

In [23]:
from huggingface_hub import login, create_repo, upload_folder
import os
import tensorflow as tf
from huggingface_hub import hf_hub_download

login(token="")

repo_name = "MNLP-HW1"
repo_id = f"tgarnier067/{repo_name}"
create_repo(repo_id, exist_ok=True, private=False)
model_folder = "models"
os.makedirs(model_folder, exist_ok=True)

models = {
    "model_ensemble": model_Ensemble
}

# Save the models in separate files
for model_name, model in models.items():
    model_file_path = os.path.join(model_folder, f"{model_name}.keras")
    model.save(model_file_path)

readme_content = """# MNLP HW1 Models
This repository contains multiple models trained for different tasks. Each model is saved as a separate `.keras` file.
"""

with open(os.path.join(model_folder, "README.md"), "w") as f:
    f.write(readme_content)

upload_folder(
    repo_id=repo_id,
    folder_path=model_folder,
    path_in_repo=""
)

print(f"All models have been successfully uploaded to Hugging Face in the repo {repo_id}!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/hf_api.py:9561: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


All models have been successfully uploaded to Hugging Face in the repo tgarnier067/MNLP-HW1!


# CNN model only

In [25]:
from tensorflow.keras import models
from tensorflow.keras import layers

# Prepare the data
train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = logits_train_model_photos
dev_vectors = logits_dev_model_photos

# Build early stop and reduce lr conditions
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Build the model, and fit
model_photos = models.Sequential()

model_photos.add(layers.Input(shape=(3,), name='input_layer'))

model_photos.add(layers.Dense(3, activation='softmax'))

model_photos.compile(optimizer=Adam(learning_rate=1e-3),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])



history = model_photos.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.1148 - loss: 3.2517 - val_accuracy: 0.2933 - val_loss: 1.8858 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1228 - loss: 2.4113 - val_accuracy: 0.3000 - val_loss: 1.5027 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2565 - loss: 1.6489 - val_accuracy: 0.2500 - val_loss: 1.2222 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3992 - loss: 1.1234 - val_accuracy: 0.3233 - val_loss: 1.1589 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5725 - loss: 0.9074 - val_accuracy: 0.5100 - val_loss: 1.1825 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6165 - loss: 0.8190 - val_accuracy: 0.5100 - val_loss: 1.2208 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6320 - loss: 0.7769 - val_ac

In [26]:
# Prepare the data for the evaluation
y_train_pred = model_photos.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_photos.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# Compute the metrics
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)
train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(y_train_true_classes, y_train_pred_classes, average='weighted')
dev_precision, dev_recall, dev_f1, _ = precision_recall_fscore_support(y_dev_true_classes, y_dev_pred_classes, average='weighted')

# Print
print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Train precision: {train_precision * 100:.2f}%")
print(f"Train recall   : {train_recall * 100:.2f}%")
print(f"Train F1-score : {train_f1 * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")
print(f"Dev precision  : {dev_precision * 100:.2f}%")
print(f"Dev recall     : {dev_recall * 100:.2f}%")
print(f"Dev F1-score   : {dev_f1 * 100:.2f}%")



196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Train accuracy : 65.19%
Train precision: 80.10%
Train recall   : 65.19%
Train F1-score : 65.77%
Dev accuracy   : 51.33%
Dev precision  : 51.43%
Dev recall     : 51.33%
Dev F1-score   : 49.12%


# Description model only

In [27]:
# Prepare the data
train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = logits_train_model_description
dev_vectors = logits_dev_model_descriptions

# Build early stop and reduce lr conditions
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Build the model, and fit it
model_description = models.Sequential()

model_description.add(layers.Input(shape=(3,), name='input_layer'))

model_description.add(layers.Dense(3, activation='softmax'))

model_description.compile(optimizer=Adam(learning_rate=1e-3),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])



history = model_description.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.2008 - loss: 1.4769 - val_accuracy: 0.3500 - val_loss: 1.0937 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6099 - loss: 0.9116 - val_accuracy: 0.6867 - val_loss: 0.8748 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8152 - loss: 0.6372 - val_accuracy: 0.6900 - val_loss: 0.8031 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8553 - loss: 0.4891 - val_accuracy: 0.6900 - val_loss: 0.7865 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8849 - loss: 0.4128 - val_accuracy: 0.6867 - val_loss: 0.7900 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8860 - loss: 0.3730 - val_accuracy: 0.6900 - val_loss: 0.8007 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8908 - loss: 0.3469 - val_ac

In [28]:
# Prepare the data for the evaluation
y_train_pred = model_description.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_description.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# Evaluation metrics
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)
train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(y_train_true_classes, y_train_pred_classes, average='weighted')
dev_precision, dev_recall, dev_f1, _ = precision_recall_fscore_support(y_dev_true_classes, y_dev_pred_classes, average='weighted')

# Print
print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Train precision: {train_precision * 100:.2f}%")
print(f"Train recall   : {train_recall * 100:.2f}%")
print(f"Train F1-score : {train_f1 * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")
print(f"Dev precision  : {dev_precision * 100:.2f}%")
print(f"Dev recall     : {dev_recall * 100:.2f}%")
print(f"Dev F1-score   : {dev_f1 * 100:.2f}%")

196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Train accuracy : 84.91%
Train precision: 86.10%
Train recall   : 84.91%
Train F1-score : 85.10%
Dev accuracy   : 69.00%
Dev precision  : 70.77%
Dev recall     : 69.00%
Dev F1-score   : 69.57%


# Features model only

In [29]:
# Prepare the data
train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = logits_train_model_features
dev_vectors = logits_dev_model_features

# Build early stop and reduce lr condition
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Build and fit the model
model_features = models.Sequential()

model_features.add(layers.Input(shape=(3,), name='input_layer'))

model_features.add(layers.Dense(3, activation='softmax'))

model_features.compile(optimizer=Adam(learning_rate=1e-3),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

history = model_features.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.5359 - loss: 0.9870 - val_accuracy: 0.5967 - val_loss: 0.8777 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6648 - loss: 0.7814 - val_accuracy: 0.5967 - val_loss: 0.7327 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6778 - loss: 0.6817 - val_accuracy: 0.6267 - val_loss: 0.6664 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7067 - loss: 0.6325 - val_accuracy: 0.7333 - val_loss: 0.6360 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7733 - loss: 0.6013 - val_accuracy: 0.7467 - val_loss: 0.6191 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7747 - loss: 0.5817 - val_accuracy: 0.7400 - val_loss: 0.6120 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7669 - loss: 0.5786 - val_ac

In [30]:
y_train_pred = model_features.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_features.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# Evaluation metrics
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)
train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(y_train_true_classes, y_train_pred_classes, average='weighted')
dev_precision, dev_recall, dev_f1, _ = precision_recall_fscore_support(y_dev_true_classes, y_dev_pred_classes, average='weighted')

# Print
print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Train precision: {train_precision * 100:.2f}%")
print(f"Train recall   : {train_recall * 100:.2f}%")
print(f"Train F1-score : {train_f1 * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")
print(f"Dev precision  : {dev_precision * 100:.2f}%")
print(f"Dev recall     : {dev_recall * 100:.2f}%")
print(f"Dev F1-score   : {dev_f1 * 100:.2f}%")

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Train accuracy : 76.96%
Train precision: 76.58%
Train recall   : 76.96%
Train F1-score : 76.54%
Dev accuracy   : 75.67%
Dev precision  : 75.42%
Dev recall     : 75.67%
Dev F1-score   : 75.27%
